# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
import sys
# workspace Session Control Class
import workspace_utils as ws
#SQL Database
from sqlalchemy import create_engine
# NLP
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#ML
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, make_scorer
#Pipeline, and features
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
#Pickle
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
database_filepath = "DisasterResponse.db"
engine = create_engine('sqlite:///' + database_filepath)
engine.table_names()

['Messages']

In [3]:
df = pd.read_sql_table('Messages',con = engine ) 

In [4]:
X = df['message']
Y = df.iloc[:,4:]
category_names = Y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


In [7]:
# Pipiline first version
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize))
    , ('tfidf', TfidfTransformer())
    , ('clf', MultiOutputClassifier(RandomForestClassifier()))])

pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
model = pipeline.fit(X_train, y_train)

y_prediction_train = model.predict(X_train)
y_prediction = model.predict(X_test)
print(classification_report(y_test.values, y_prediction, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      4091
               request       0.83      0.40      0.54       877
                 offer       0.00      0.00      0.00        28
           aid_related       0.73      0.52      0.61      2185
          medical_help       0.58      0.07      0.12       427
      medical_products       0.44      0.03      0.05       271
     search_and_rescue       0.62      0.09      0.16       141
              security       1.00      0.01      0.02       101
              military       0.77      0.05      0.10       196
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.15      0.26       321
                  food       0.81      0.31      0.45       586
               shelter       0.90      0.27      0.42       427
              clothing       0.29      0.03      0.05        78
                 money       0.50      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [10]:
print(classification_report(y_test.values, y_prediction, target_names=category_names))
accuracy = (y_prediction == y_test).mean()
print("Accuracy:\n", accuracy)

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      4091
               request       0.83      0.40      0.54       877
                 offer       0.00      0.00      0.00        28
           aid_related       0.73      0.52      0.61      2185
          medical_help       0.58      0.07      0.12       427
      medical_products       0.44      0.03      0.05       271
     search_and_rescue       0.62      0.09      0.16       141
              security       1.00      0.01      0.02       101
              military       0.77      0.05      0.10       196
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.15      0.26       321
                  food       0.81      0.31      0.45       586
               shelter       0.90      0.27      0.42       427
              clothing       0.29      0.03      0.05        78
                 money       0.50      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {
    'vect__ngram_range': ((1, 2), (1, 3),(1,4))
    , 'vect__max_df': (0.5, 0.75, 1.0)
    , 'tfidf__use_idf': (True, False)
    #, 'clf__estimator__n_estimators': [50, 100, 200]
    , 'clf__estimator__min_samples_split': [2, 4]
}

cv = GridSearchCV(pipeline, param_grid = parameters)

In [12]:
with ws.active_session():
    cv_model = cv.fit(X_train, y_train)

    y_cv_prediction = cv_model.predict(X_test)

    # Print classification report on test data
    print(classification_report(y_test.values, y_cv_prediction, target_names=category_names))

    

                        precision    recall  f1-score   support

               related       0.84      0.90      0.87      4091
               request       0.86      0.41      0.56       877
                 offer       0.00      0.00      0.00        28
           aid_related       0.78      0.45      0.58      2185
          medical_help       0.80      0.06      0.11       427
      medical_products       0.70      0.05      0.10       271
     search_and_rescue       0.53      0.06      0.10       141
              security       0.00      0.00      0.00       101
              military       0.75      0.02      0.03       196
           child_alone       0.00      0.00      0.00         0
                 water       0.79      0.36      0.49       321
                  food       0.82      0.46      0.59       586
               shelter       0.85      0.31      0.46       427
              clothing       0.50      0.03      0.05        78
                 money       0.50      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [13]:
print('Best Parameters:', cv.best_params_)

Best Parameters: {'clf__estimator__min_samples_split': 2, 'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 3)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
tuned_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize,max_df=0.5,ngram_range=(1,3)))
    , ('tfidf', TfidfTransformer(use_idf=True))
    , ('clf', MultiOutputClassifier(RandomForestClassifier(min_samples_split=2)))])
model = tuned_pipeline
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test.values, y_pred, target_names=category_names))


                        precision    recall  f1-score   support

               related       0.83      0.92      0.87      4049
               request       0.84      0.41      0.55       934
                 offer       0.00      0.00      0.00        26
           aid_related       0.77      0.44      0.56      2209
          medical_help       0.64      0.09      0.15       430
      medical_products       0.74      0.11      0.19       244
     search_and_rescue       0.73      0.06      0.10       143
              security       0.00      0.00      0.00       100
              military       0.40      0.01      0.02       172
           child_alone       0.00      0.00      0.00         0
                 water       0.83      0.26      0.39       328
                  food       0.82      0.41      0.55       605
               shelter       0.80      0.19      0.31       467
              clothing       0.80      0.11      0.19        76
                 money       0.80      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

Different machine learning model (Random Forest, KNN, ADaboost, and Linear SVC) have been tested and among them Linear SVC perfomed better results for Precision, recall and f1 scores. In the following we will run GridSearch CV on the SVC model to find the best parameters.

In [14]:
MOC = MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state=0)))
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize,)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MOC)
    ])


In [15]:
f1 = make_scorer(f1_score , average='macro')
parameters = {
    
    'features__text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__estimator__multi_class': ('ovr','crammer_singer'),
    'clf__estimator__estimator__C': [1, 2, 5],
    'features__text_pipeline__vect__ngram_range':((1, 3), (1, 4),(1,5))
   
}

    
cv = GridSearchCV(pipeline,param_grid=parameters,verbose=1,scoring=f1)

In [16]:
with ws.active_session():
    cv_fitted = cv.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/sit

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/sit

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/sit

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/sit

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/sit

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/sit

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/sit

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/sit

In [17]:
y_prediction = cv_fitted.predict(X_test)

In [18]:
print(classification_report(y_test.values, y_prediction, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.86      0.94      0.90      4091
               request       0.68      0.71      0.69       877
                 offer       0.00      0.00      0.00        28
           aid_related       0.65      0.82      0.73      2185
          medical_help       0.55      0.43      0.48       427
      medical_products       0.57      0.35      0.43       271
     search_and_rescue       0.54      0.22      0.31       141
              security       0.20      0.01      0.02       101
              military       0.61      0.41      0.49       196
           child_alone       0.00      0.00      0.00         0
                 water       0.61      0.79      0.69       321
                  food       0.72      0.80      0.76       586
               shelter       0.66      0.73      0.70       427
              clothing       0.64      0.44      0.52        78
                 money       0.53      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [19]:
# print best parameter after tuning 
print('Best Parameters:',cv.best_params_) 
  

Best Parameters: {'clf__estimator__estimator__C': 5, 'clf__estimator__estimator__multi_class': 'crammer_singer', 'features__text_pipeline__tfidf__use_idf': False, 'features__text_pipeline__vect__ngram_range': (1, 5)}


## Final Model

In [11]:
Tuned_MOC = MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state=0, multi_class='crammer_singer',C = 5)))
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize,ngram_range=(1,5))),
                ('tfidf', TfidfTransformer(use_idf=False))
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', Tuned_MOC)
    ])
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test.values, y_pred, target_names=category_names))

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


                        precision    recall  f1-score   support

               related       0.85      0.94      0.90      4049
               request       0.69      0.72      0.71       934
                 offer       0.00      0.00      0.00        26
           aid_related       0.64      0.83      0.73      2209
          medical_help       0.49      0.43      0.46       430
      medical_products       0.49      0.44      0.46       244
     search_and_rescue       0.49      0.17      0.26       143
              security       0.20      0.01      0.02       100
              military       0.51      0.42      0.46       172
           child_alone       0.00      0.00      0.00         0
                 water       0.61      0.79      0.69       328
                  food       0.71      0.80      0.75       605
               shelter       0.67      0.67      0.67       467
              clothing       0.72      0.50      0.59        76
                 money       0.60      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [10]:
model = pipeline
pickle.dump(model, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.